In [1]:
import pandas as pd
import os, cv2, keras
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
from keras import layers

In [3]:
df = pd.read_csv('cropped/hiragana_labels.csv')

In [4]:
X_=[]
y_=[]
for image_id, labels in df.values:
    image_fn = 'cropped/hiragana_cropped/{}.jpg'.format(image_id)
    image = Image.open(image_fn)
    image = np.asarray(image)
    X_.append(image)
    y_.append(labels)

In [5]:
X_ = np.array(X_)
y_ = np.array(y_)

In [6]:
unique, counts = np.unique(y_, return_counts=True)

In [7]:
i = 0
classes = []
for c in counts:
    if(c >= 10 and unique[i] != 'ゑ' and unique[i] != 'ゐ'):
        cl = unique[i]
        classes.append(cl)
    i+=1

In [8]:
j = 0
yy1_ = []
XX1_ = []
for y in y_:
    if(y in classes):
        yy1_.append(y)
        XX1_.append(X_[j])
    j+= 1

In [9]:
X_ = np.array(XX1_)
y_ = np.array(yy1_)

In [10]:
print(X_.shape)
print(y_.shape)

(184700, 100, 100)
(184700,)


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, BatchNormalization, MaxPool2D, MaxPooling2D, AveragePooling2D, Input
from keras.optimizers import Adam
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint
from datetime import datetime

from keras import backend as K

In [12]:
leinst = LabelEncoder()
yyinst = leinst.fit_transform(y_)

# x_train, x_test, y_train, y_test = train_test_split(X_, yyinst, test_size=0.3, shuffle=True, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X_, yyinst, test_size=0.2, shuffle=True, stratify=yyinst)

# x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size=0.33, shuffle=True, random_state=42)
x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, stratify=y_test)

# x_train = x_train.reshape(len(x_train),128,44,1)
# x_test = x_test.reshape(len(x_test),128,44,1)
# x_validation = x_validation.reshape(len(x_validation),128,44,1)
print(x_train.shape)
print(x_test.shape)
print(x_validation.shape)


num_classes = len(leinst.classes_)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_validation = to_categorical(y_validation, num_classes=num_classes)

(147760, 100, 100)
(18470, 100, 100)
(18470, 100, 100)


In [40]:
num_classes

71

In [13]:
# x_train_reshaped = x_train.reshape(x_train.shape[0], -1)

In [14]:
# np.savetxt("x_train.csv", x_train_reshaped)

In [15]:
# np.savetxt("y_train.csv", y_train)

In [16]:
# x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
# np.savetxt("x_test.csv", x_test_reshaped)

In [17]:
# np.savetxt("y_test.csv", y_test)

In [18]:
# x_validation_reshaped = x_validation.reshape(x_validation.shape[0], -1)
# np.savetxt("x_validation.csv", x_validation_reshaped)

In [19]:
# np.savetxt("y_validation.csv", y_validation)

In [ ]:
# x_train = np.loadtxt("x_train.csv")

In [ ]:
# load_original_arr = x_train.reshape(
#     loaded_arr.shape[0], loaded_arr.shape[1] // arr.shape[2], arr.shape[2])

In [13]:
K.clear_session()
def Kuzushiji_Classifier(in_):
    model_ = Conv2D(64,kernel_size=(3, 3),activation='relu')(in_)
    model_ = BatchNormalization()(model_)
    model_ = Conv2D(32,kernel_size=(3, 3),activation='relu')(model_)
    model_ = MaxPooling2D(pool_size=(2, 2))(model_)
    model_ = BatchNormalization()(model_)
    model_ = Dropout(0.4)(model_)
    
    model_ = Conv2D(32,kernel_size=(3, 3), strides=2,activation='relu')(model_)
    model_ = MaxPooling2D(pool_size=(2, 2))(model_)
    model_ = BatchNormalization()(model_)
    model_ = Dropout(0.4)(model_)
    
    model_ = Flatten()(model_)
    model_ = Dense(128, activation='relu')(model_)
    model_ = Dropout(0.4)(model_)
    model_ = Dense(num_classes, activation='softmax')(model_)
    return model_

In [14]:
Input_Sample = Input(shape=(100, 100, 1))
Output_ = Kuzushiji_Classifier(Input_Sample)
Model_Enhancer = Model(inputs=Input_Sample, outputs=Output_)

In [15]:
Model_Enhancer.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Model_Enhancer.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 98, 98, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128   

In [17]:
checkpointer = ModelCheckpoint('model_Kuzushiji1.h5', verbose=0,mode='auto', monitor='val_accuracy',save_best_only=True)

In [18]:
ModelHistory = Model_Enhancer.fit(x_train, y_train,
          batch_size=16,
          epochs=10,
          verbose=1,
          callbacks=[checkpointer],
          validation_data=(x_validation, y_validation))

Epoch 1/10
9235/9235 [==============================] - 2919s 316ms/step - loss: 1.1259 - accuracy: 0.7125 - val_loss: 0.4731 - val_accuracy: 0.8836
Epoch 2/10
9235/9235 [==============================] - 21093s 2s/step - loss: 0.6710 - accuracy: 0.8184 - val_loss: 0.3510 - val_accuracy: 0.9069
Epoch 3/10
9235/9235 [==============================] - 2895s 314ms/step - loss: 0.5737 - accuracy: 0.8434 - val_loss: 0.3676 - val_accuracy: 0.9096
Epoch 4/10
9235/9235 [==============================] - 3000s 325ms/step - loss: 0.5186 - accuracy: 0.8559 - val_loss: 0.2948 - val_accuracy: 0.9252
Epoch 5/10
9235/9235 [==============================] - 12750s 1s/step - loss: 0.4946 - accuracy: 0.8628 - val_loss: 0.2863 - val_accuracy: 0.9262
Epoch 6/10
9235/9235 [==============================] - 3125s 338ms/step - loss: 0.4700 - accuracy: 0.8697 - val_loss: 0.2774 - val_accuracy: 0.9306
Epoch 7/10
9235/9235 [==============================] - 3161s 342ms/step - loss: 0.4538 - accuracy: 0.8726 - v

In [ ]:
Model_Enhancer.load_weights('model_Kuzushiji1.h5')

In [19]:
train_score = Model_Enhancer.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: ", train_score[1])

4618/4618 [==============================] - 426s 92ms/step - loss: 0.1578 - accuracy: 0.9593
Training Accuracy:  0.9592785835266113


In [20]:
test_score = Model_Enhancer.evaluate(x_test, y_test, verbose=1)
print("Test Accuracy: ", test_score[1])

578/578 [==============================] - 56s 96ms/step - loss: 0.2475 - accuracy: 0.9340
Test Accuracy:  0.934001088142395


In [22]:
ModelHistory = Model_Enhancer.fit(x_train, y_train,
          batch_size=16,
          epochs=10,
          verbose=1,
          callbacks=[checkpointer],
          validation_data=(x_validation, y_validation))

Epoch 1/10
9235/9235 [==============================] - 2836s 307ms/step - loss: 0.4161 - accuracy: 0.8831 - val_loss: 0.2718 - val_accuracy: 0.9313
Epoch 2/10
9235/9235 [==============================] - 3113s 337ms/step - loss: 0.4090 - accuracy: 0.8851 - val_loss: 0.2561 - val_accuracy: 0.9337
Epoch 3/10
9235/9235 [==============================] - 3109s 337ms/step - loss: 0.4042 - accuracy: 0.8863 - val_loss: 0.2669 - val_accuracy: 0.9338
Epoch 4/10
9235/9235 [==============================] - 2977s 322ms/step - loss: 0.4023 - accuracy: 0.8865 - val_loss: 0.2824 - val_accuracy: 0.9306
Epoch 5/10
9235/9235 [==============================] - 2902s 314ms/step - loss: 0.3998 - accuracy: 0.8875 - val_loss: 0.2450 - val_accuracy: 0.9386
Epoch 6/10
9235/9235 [==============================] - 3001s 325ms/step - loss: 0.3984 - accuracy: 0.8870 - val_loss: 0.2470 - val_accuracy: 0.9391
Epoch 7/10
9235/9235 [==============================] - 3053s 331ms/step - loss: 0.3894 - accuracy: 0.8903

In [23]:
train_score = Model_Enhancer.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: ", train_score[1])

4618/4618 [==============================] - 414s 90ms/step - loss: 0.1250 - accuracy: 0.9670
Training Accuracy:  0.9669666886329651


In [24]:
test_score = Model_Enhancer.evaluate(x_test, y_test, verbose=1)
print("Test Accuracy: ", test_score[1])

578/578 [==============================] - 62s 106ms/step - loss: 0.2439 - accuracy: 0.9376
Test Accuracy:  0.9376285672187805


In [25]:
y_pred = Model_Enhancer.predict(x_test)

578/578 [==============================] - 53s 92ms/step


In [ ]:
y_pred[0]

array([2.77684027e-08, 2.01823216e-08, 2.54049429e-12, 8.78703360e-13,
       6.87377167e-07, 8.82464974e-07, 7.14351365e-04, 2.65569025e-08,
       3.30244188e-06, 9.83050197e-10, 7.37276496e-05, 2.91381230e-09,
       1.61631342e-05, 3.41546613e-09, 9.87379390e-05, 3.14488176e-07,
       4.70635248e-03, 4.68442153e-07, 4.83857766e-05, 1.90324170e-06,
       1.62559352e-03, 1.18288908e-05, 9.89352405e-01, 1.08182006e-08,
       1.39653427e-03, 1.85657871e-08, 9.53455979e-04, 2.97635961e-12,
       4.28336797e-08, 7.07658572e-20, 1.34399256e-10, 5.85896894e-04,
       5.07749434e-08, 9.34125346e-05, 3.00122482e-09, 6.64051186e-05,
       1.05981016e-07, 1.02808029e-09, 4.12002876e-08, 1.40354360e-11,
       3.57572499e-10, 2.33584108e-09, 1.30238723e-05, 1.43422763e-13,
       1.92964134e-12, 1.78013837e-09, 1.47374907e-14, 8.74781220e-11,
       3.57715635e-07, 9.89595450e-13, 6.56389831e-09, 5.67298048e-05,
       2.56915040e-13, 2.27001733e-11, 1.12018519e-04, 1.63481015e-08,
      

In [30]:
i = np.argmax(y_pred[5])
print("Prediction: ", leinst.classes_[i])

Prediction:  て


In [31]:
i = np.argmax(y_test[5])
print("Prediction: ", leinst.classes_[i])

Prediction:  て


In [32]:
from sklearn.metrics import f1_score

In [33]:
y_true = np.argmax(y_test, axis=1)

In [34]:
y_pr = np.argmax(y_pred, axis=1)

In [1]:
f1 = f1_score(y_true, y_pr, average=None)

NameError: name 'f1_score' is not defined

In [36]:
from sklearn.metrics import classification_report

In [37]:
n = 0
labels = []
for clas in leinst.classes_:
    labels.append(n)
    # print(clas, n)
    n += 1

In [38]:
report = classification_report(y_true, y_pr, labels=labels, target_names=leinst.classes_)
report_dict = classification_report(y_true, y_pr, labels=labels, target_names=leinst.classes_, output_dict=True)

C:\Users\marij\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\marij\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\marij\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [39]:
print(report)

              precision    recall  f1-score   support

           あ       0.96      0.94      0.95       308
           い       0.95      0.98      0.96       415
           う       0.97      0.94      0.95       260
           え       0.92      0.89      0.91        38
           お       0.94      0.94      0.94       209
           か       0.91      0.85      0.88       628
           が       0.90      0.95      0.92       272
           き       0.94      0.94      0.94       355
           ぎ       0.71      0.77      0.74        26
           く       0.95      0.95      0.95       416
           ぐ       0.94      0.68      0.79        22
           け       0.91      0.85      0.88       207
           げ       0.76      0.59      0.67        27
           こ       0.94      0.86      0.90       259
           ご       0.89      0.76      0.82        42
           さ       0.97      0.91      0.94       305
           ざ       0.85      0.78      0.82        60
           し       0.91    